# Node connectivity information
This example demonstrates some functionality related to node connectivity, such as:
- Counting number of links connected to a node
- Coutning number of enabled links connected to a node
- Getting a list of upstream/downstream links connected to a node

In [ ]:
import pandas as pd
from mikeplus import Database

In [ ]:
db = Database("../tests/testdata/Db/Sirius/Sirius.sqlite")
db

In [ ]:
# Get alias to msm_Node table for convenience
msm_node = db.tables.msm_Node
msm_node

In [ ]:
# Get all node MUIDs
muids = msm_node.get_muids()
muids

In [ ]:
# Get number of links connected to Node_33
msm_node.get_number_of_links("Node_33")

In [ ]:
# Get number of links connected to Node_33 that are enabled
msm_node.get_number_of_links_enabled("Node_33")

In [ ]:
# Get link MUIDs where "Node_33" is the upstream/from node.
upstream_links = msm_node.get_upstream_links("Node_33")
upstream_links

In [ ]:
# Confirm that the MUIDs make sense with a query
db.tables.msm_Link.select(["FromNodeID", "ToNodeID"]).where("FromNodeID = 'Node_36'").to_pandas()

In [ ]:
# Get link MUIDs where "Node_33" is the downstream/to node.
downstream_links = msm_node.get_downstream_links("Node_33")
downstream_links

In [ ]:
# Confirm that the MUIDs make sense with a query
db.tables.msm_Link.select(["FromNodeID", "ToNodeID"]).where("ToNodeID = 'Node_33'").to_pandas()

In [ ]:
# Same as above, but for all nodes and visualized in a DataFrame
data = {
    muid: {
        "num_of_links": msm_node.get_number_of_links(muid),
        "num_of_links_enabled": msm_node.get_number_of_links_enabled(muid),
        "num_upstream_links": len(msm_node.get_upstream_links(muid)),
        "num_downstream_links": len(msm_node.get_downstream_links(muid)),
    }
    for muid in msm_node.get_muids()
}
df = pd.DataFrame(data).T
df

In [ ]:
# Close the database
db.close()